In [ ]:
pip install spotipy lyricsgenius

In [ ]:
import spotipy
import lyricsgenius
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="57145e04c4e34b998712d9eede6aaf7d",
                                                           client_secret="0c20611c261b4c16b407f954d53ea48e"))

results = sp.search(q='drake', limit=3)
for idx, track in enumerate(results['tracks']['items']):
    print(idx, track['name'])

In [ ]:
genius = lyricsgenius.Genius("wJRVTKvwP6Co2RwJEfiVvox_QcoPEm23z3hst8BOfHPa9qkF4Fuj6jQuRfsI5_rn")
artist = genius.search_artist("drake", max_songs=3, sort="title")
print(artist.songs)

In [ ]:
song = genius.search_song("Marvin's Room", artist.name)
print(song.lyrics)

stuff added by bing below

In [13]:
import csv

csvfile = open('tcc_ceds_music.csv', newline='', encoding='utf-8')
reader = csv.reader(csvfile, delimiter=',')
i = 0
for row in reader:
  genre = row[4]
  lyrics = row[5]
  # print(genre, lyrics)
  i += 1
  # if i == 5:
  #   break
print(i)

28373
